In [ ]:
import os

In [ ]:
rawDir = '../../../Data-Lake/02-Raw-Zone/uk-housing'
structDir = '../../../Data-Lake/03-Structured-Zone/uk-housing'

In [ ]:
if not os.path.exists(structDir):
    os.makedirs(structDir)

In [ ]:
import pandas as pd

In [ ]:
def readDF(rawDir, filename):
    fileFullName = os.path.join(rawDir,filename)
    df = pd.read_csv(fileFullName,header=0)
    df2=df[['3','2','1','1']].copy(deep=True)
    df2.columns = ['postcode','date','price','Outstanding']
    return df2

In [ ]:
from mortgage import Loan
import math as m
from datetime import datetime as dt 
import datetime as d
from dateutil import relativedelta

In [ ]:
def calculate_outstanding(row):
    p = m.floor(row['price'])
    loan = Loan(principal=p, interest=0.05, term=25)
    datenow = str(d.date.today().strftime("%Y-%m-%d"))
    dbuy=str(row['date'])
    date1 = dt.strptime(dbuy, '%Y-%m-%d %H:%M')
    date2 = dt.strptime(datenow, '%Y-%m-%d')
    r = relativedelta.relativedelta(date2, date1)
    v = loan.total_paid - (loan.monthly_payment * ((r.years * 12) + r.months))
    if v < 0:
        v = 0
    return v

In [ ]:
for dirName, subdirList, fileList in os.walk(rawDir):
    print('Found directory: %s' % dirName)
    for fname in fileList:
        filename, file_extension = os.path.splitext(fname)
        if filename[:3] == 'pp-':
            print('>>', filename)
            df = readDF(rawDir, fname)
            df.loc[df.index.tolist(),'Outstanding'] = df.apply(calculate_outstanding, axis=1)
            filestructname = os.path.join(structDir,fname)
            df.to_csv(filestructname, header=True)